# Assignment - Segmenting and Clustering Neighborhoods in Toronto

<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li style="margin-bottom: 10px;"><a href="df">Web scraping and create a dataframe for Toronto neighbourhoods</a></li>
        <li style="margin-bottom: 10px;"><a href="ll">Get the latitude and the longitude coordinates of each neighborhood</a></li>
        <li style="margin-bottom: 10px;"><a href="clu">Explore and cluster the neighborhoods in Toronto</a></li>
     </ol>
    
</div>

<h2 id="df">1. Web scraping and create a dataframe for Toronto neighbourhoods</h2>